In [1]:
#!curl -L "https://www.microsoft.com/en-us/download/confirmation.aspx?id=54765" -o "/tmp/cats-and-dogs.zip"
!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip" \
    -O '/tmp/cats-and-dogs.zip'

--2022-11-05 16:13:32--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 2.18.232.239, 2a02:26f0:6c00:29b::317f, 2a02:26f0:6c00:2a9::317f
Connecting to download.microsoft.com (download.microsoft.com)|2.18.232.239|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘/tmp/cats-and-dogs.zip’

/tmp/cats-and-dogs. 100%[===================>] 786.67M  34.0MB/s    in 22s     

2022-11-05 16:13:54 (36.5 MB/s) - ‘/tmp/cats-and-dogs.zip’ saved [824887076/824887076]



In [10]:
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile

In [11]:
base_dir = '/tmp/cats-and-dogs/'

local_zip = "/tmp/cats-and-dogs.zip"
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall(base_dir)
zip_ref.close()

In [12]:
print("Cat Pictures: " + str(len(os.listdir(base_dir + 'PetImages/Cat/'))))
print("Dog Pictures: " + str(len(os.listdir(base_dir + 'PetImages/Dog/'))))

Cat Pictures: 12501
Dog Pictures: 12501


In [13]:
try:
    os.mkdir(base_dir + 'training')
    os.mkdir(base_dir + 'training/cats')
    os.mkdir(base_dir + 'training/dogs')
    
    os.mkdir(base_dir + 'testing')
    os.mkdir(base_dir + 'testing/cats')
    os.mkdir(base_dir + 'testing/dogs')
except OSError:
    pass

In [20]:
def copy_pics(SOURCE_FOLDER, DESTINATION_FOLDER, SET):
    for filename in SET:
        this_file = SOURCE_FOLDER + filename
        destination = DESTINATION_FOLDER + filename
        copyfile(this_file, destination)

In [23]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is Zero length, so ignoring.")
    
    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    
    shuffled_set = random.sample(files, len(files))
    
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[-testing_length:]
    
    copy_pics(SOURCE, TRAINING, training_set)
    copy_pics(SOURCE, TESTING, testing_set)
    
    print("SOURCE: " + SOURCE + " len:" + str(len(os.listdir(SOURCE))))
    print("TRAINING: " + TRAINING + " len:" + str(len(os.listdir(TRAINING))))
    print("TESTING: " + TESTING + " len:" + str(len(os.listdir(TESTING))))
    
    
    

In [25]:
CAT_SOURCE_DIR = base_dir + 'PetImages/Cat/'
TRAINING_CATS_DIR = base_dir + 'training/cats/'
TESTING_CATS_DIR = base_dir + 'testing/cats/'

DOG_SOURCE_DIR = base_dir + 'PetImages/Dog/'
TRAINING_DOGS_DIR = base_dir + 'training/dogs/'
TESTING_DOGS_DIR = base_dir + 'testing/dogs/'

In [28]:
split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

666.jpg is Zero length, so ignoring.
SOURCE: /tmp/cats-and-dogs/PetImages/Cat/ len:12501
TRAINING: /tmp/cats-and-dogs/training/cats/ len:12500
TESTING: /tmp/cats-and-dogs/testing/cats/ len:3391
11702.jpg is Zero length, so ignoring.
SOURCE: /tmp/cats-and-dogs/PetImages/Dog/ len:12501
TRAINING: /tmp/cats-and-dogs/training/dogs/ len:12500
TESTING: /tmp/cats-and-dogs/testing/dogs/ len:3382


In [32]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [33]:
model.compile(optimizer=RMSprop(learning_rate=0.001),loss='binary_crossentropy', metrics=['accuracy'])

In [34]:
TRAINING_DIR = base_dir + 'training/'
train_datagen = ImageDataGenerator(rescale=1.0/255.)
train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    batch_size=250,
    class_mode='binary',
    target_size=(150, 150)
)

Found 24998 images belonging to 2 classes.


In [36]:
VALIDATION_DIR = base_dir + 'testing/'
validation_datagen = ImageDataGenerator(rescale=1.0/255.)
validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    batch_size=250,
    class_mode='binary',
    target_size=(150, 150)
)

Found 6773 images belonging to 2 classes.


In [37]:
history = model.fit(
    train_generator, 
    epochs=15, 
    steps_per_epoch=90, 
    validation_data=validation_generator, 
    validation_steps=10
)

Epoch 1/15
89/90 [============================>.] - ETA: 2s - loss: 0.7143 - accuracy: 0.5681

/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:850: UserWarning: Truncated File Read
  warnings.warn(str(msg))


90/90 [==============================] - 220s 2s/step - loss: 0.7133 - accuracy: 0.5690 - val_loss: 0.6321 - val_accuracy: 0.6392
Epoch 2/15
90/90 [==============================] - 217s 2s/step - loss: 0.6008 - accuracy: 0.6751 - val_loss: 0.5723 - val_accuracy: 0.6912
Epoch 3/15
90/90 [==============================] - 217s 2s/step - loss: 0.5384 - accuracy: 0.7265 - val_loss: 0.5056 - val_accuracy: 0.7632
Epoch 4/15
90/90 [==============================] - 216s 2s/step - loss: 0.4993 - accuracy: 0.7518 - val_loss: 0.5457 - val_accuracy: 0.7188
Epoch 5/15
90/90 [==============================] - 217s 2s/step - loss: 0.4688 - accuracy: 0.7761 - val_loss: 0.4395 - val_accuracy: 0.8008
Epoch 6/15
90/90 [==============================] - 218s 2s/step - loss: 0.4498 - accuracy: 0.7860 - val_loss: 0.3998 - val_accuracy: 0.8248
Epoch 7/15
90/90 [==============================] - 216s 2s/step - loss: 0.4130 - accuracy: 0.8110 - val_loss: 0.3362 - val_accuracy: 0.8620
Epoch 8/15
90/90 [======

In [42]:
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs=range(len(acc))

plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")

plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.figure()


TypeError: 'History' object is not subscriptable

In [ ]:
#google colab test using upload
import numpyy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn inuploadedoadedploaded.keys():
    path = '/content/' + fn
    img = image.load_img(path, target_size(150, 15imagege
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    print(classes[0])
    if classes[0]>0.5:
        print(fn + " is a dog")
    else:
        print(fn + " is a cat")